In [1]:
import subprocess
from time import sleep

import numpy as np
import pynvml
from tqdm import tqdm

pynvml.nvmlInit()


def get_free_MB(i):
    handle = pynvml.nvmlDeviceGetHandleByIndex(i)
    return pynvml.nvmlDeviceGetMemoryInfo(handle).free / 1024 / 1024


def get_avail_gpu(size=10000):
    num = pynvml.nvmlDeviceGetCount()
    free = []
    for _ in range(50):
        sleep(0.4)
        free.append([get_free_MB(i) for i in range(num)])
    free = np.array(free).T.tolist()
    for i in range(num):
        if min(free[i]) > size:
            return i
    return None

In [2]:
note = "switch_gan_front"
tasks = []

for gan in [2, 4, 6, 8, 10]:
    for tol in [30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]:
        tasks.append(
            f"switch_gan{gan:02d}_tol{tol}_front{10-gan:02d}",
        )

In [3]:
for task in tqdm(tasks):
    while (gpu := get_avail_gpu()) is None:
        pass
    print(f"python train.py --train {task} --note {note} -g {gpu}")
    subprocess.Popen(f"python train.py --train {task} --note {note} -g {gpu}", shell=True)

  1%|▏         | 1/70 [00:20<23:19, 20.28s/it]

python train.py --train switch_gan02_tol30_front08 --note switch_gan_front -g 6


  3%|▎         | 2/70 [00:40<22:55, 20.22s/it]

python train.py --train switch_gan02_tol35_front08 --note switch_gan_front -g 6


  4%|▍         | 3/70 [01:00<22:32, 20.18s/it]  2.06s/it]

...

 24%|██▍       | 17/70 [1:00:10<2:06:58, 143.74s/it]s/it]

python train.py --train switch_gan04_tol40_front06 --note switch_gan_front -g 6


training:   2%|▏         | 7/400 [00:24<21:25,  3.27s/it]]Traceback (most recent call last):
  File "/home/zhangyixiang/wfp/train.py", line 94, in <module>
    attack.train(
  File "/home/zhangyixiang/wfp/attack/base.py", line 115, in train
    net.to(device)
  File "/home/zhangyixiang/miniconda3/envs/wfp/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1152, in to
    return self._apply(convert)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/zhangyixiang/miniconda3/envs/wfp/lib/python3.12/site-packages/torch/nn/modules/module.py", line 802, in _apply
    module._apply(fn)
  File "/home/zhangyixiang/miniconda3/envs/wfp/lib/python3.12/site-packages/torch/nn/modules/module.py", line 802, in _apply
    module._apply(fn)
  File "/home/zhangyixiang/miniconda3/envs/wfp/lib/python3.12/site-packages/torch/nn/modules/module.py", line 825, in _apply
    param_applied = fn(param)
                    ^^^^^^^^^
  File "/home/zhangyixiang/miniconda3/envs/wfp/lib/python3.12/site-packages

python train.py --train switch_gan04_tol45_front06 --note switch_gan_front -g 6


training:  70%|██████▉   | 279/400 [14:26<04:09,  2.06s/it]